# Zurich Instruments LabOne Python API Example
# Connect to a Zurich Instruments device

Demonstrate how to connect to a Zurich Instruments device via the Data Server
program. Create an API session by connecting to a Zurich Instruments device via
the Data Server.

Identify the device and check consistency across the LabOne stack.

> Note:
>
> This example works with all Zurich Instruments devices except the HF2LI. A
> separate example `example_connect` in the hf2 directory exists for the HF2LI.

Requirements:
* LabOne Version >= 20.02
* Instruments:
    1 x Zurich Instruments device

---

In [ ]:
import zhinst.ziPython

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server. (The MFLI/IA devices
have a Data Server running on the device itself, please see the
[user manual](https://docs.zhinst.com/mfli_user_manual/getting_started/device_connectivity_instrument.html)
for detailed explanation.)

In [ ]:
device_id = "dev2345" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection or when MFLI/MFIA is connected to a remote Data Server.
#interface = "USB" # For all instruments connected to the host computer via USB except MFLI/MFIA.
#interface = "PCIe" # For MFLI/MFIA devices in case the Data Server runs on the device.

server_host = "localhost"
server_port = 8004
api_level = 6 # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.ziPython.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

Identify the connected device by trigger a blinking of the power led for a few seconds.

In [ ]:
daq.setInt(f"/{device_id}/system/identify", 1)

The versions of LabOne and the zhinst.ziPython module should match to ensure
deterministic behaviour.

In [ ]:
from zhinst.ziPython import __version__ as zhinst_version

if daq.revision() == daq.getInt("/zi/about/revision"):
    print("LabOne and zhinst.ziPython version match.")
else:
    labone_version = daq.getString("/zi/about/version")
    labone_revision = daq.getInt("/zi/about/revision")
    print(
        f"zhinst.ziPython version ({zhinst_version}) does not match",
        f"the LabOne version ({labone_version}.{str(labone_revision)[4:]})!",
    )


The firmware revision should match the LabOne version to ensure deterministic behavior.

In [ ]:
import json
status_flag = json.loads(daq.getString("/zi/devices"))[device_id.upper()]["STATUSFLAGS"]
if status_flag & 1 << 8:
    print(
        "The device is currently updating please try again after the update "
        "process is complete"
    )
elif status_flag & 1 << 4 or status_flag & 1 << 5:
    print(
        "The firmware does not match the LabOne version. "
        "Please update the firmware, e.g. in the LabOne UI."
    )
elif status_flag & 1 << 6 or status_flag & 1 << 7:
    print(
        "The firmware does not match the LabOne version. "
        "Please update LabOne to the latest version from "
        "https://www.zhinst.com/support/download-center."
    )
else:
    print("The Firmware matches the LabOne version")